### 1. Carga de librerias

In [65]:
import pandas as pd

### 2. Carga de files

In [66]:
df_sales_original = pd.read_csv('C:\\Users\\gaspi\\Desktop\\TFM\\item_sales.csv')

In [67]:
df_prices_original = pd.read_csv('C:\\Users\\gaspi\\Desktop\\TFM\\item_prices.csv')

In [68]:
df_calendar_original = pd.read_csv('C:\\Users\\gaspi\\Desktop\\TFM\\daily_calendar_with_events.csv')

In [69]:
df_prices = df_prices_original.copy()

In [70]:
df_calendar = df_calendar_original.copy()

## 3. Exploracion de dataframes

#### 3.1. df_calendar

Se vio:
- Columnas que coincidian con los demas df, y se encontro que matchea la columna "d" con df_sales.
- Fiestas, y se vio que se podrian agregar Navidad y 4 de Julio.


In [71]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [72]:
df_calendar

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,NaN
1909,2016-04-21,Thursday,6,d_1910,NaN
1910,2016-04-22,Friday,7,d_1911,NaN
1911,2016-04-23,Saturday,1,d_1912,NaN


In [73]:
calendar_events = df_calendar.dropna()
events = calendar_events['event'].unique()
events

array(['SuperBowl', 'Ramadan starts', 'Thanksgiving', 'NewYear', 'Easter'],
      dtype=object)

#### 3.2. df_sales

- Se pueden pasar las columnas "d_" a filas para poder analizar la informacion
- se puede agregar los datos del calendar ya que los datos "d_" coinciden con el df_calendar
- Para poder agregar el precio habria que agregar una columna con "yearweek" a partir de la fecha y poder matchear con el df_prices
- Identificacion de valores unicos de items, categories, departamentos, tiendas y ciudades

In [74]:
df_sales_original

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,2,...,2,0,0,0,0,0,1,0,0,1
30486,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,6,0,...,2,1,0,2,0,1,0,0,1,0
30488,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [75]:
df_sales_description = df_sales_original.iloc[:, :7]


In [76]:
df_sales_description.nunique()

id            30490
item           3049
category          3
department        7
store            10
store_code       10
region            3
dtype: int64

In [77]:
items  = df_sales_original['item'].unique()
print(items)

['ACCESORIES_1_001' 'ACCESORIES_1_002' 'ACCESORIES_1_003' ...
 'SUPERMARKET_3_825' 'SUPERMARKET_3_826' 'SUPERMARKET_3_827']


In [78]:
categories = df_sales_original['category'].unique()
print(categories)

['ACCESORIES' 'HOME_&_GARDEN' 'SUPERMARKET']


In [79]:
departments = df_sales_original['department'].unique()
print(departments)

['ACCESORIES_1' 'ACCESORIES_2' 'HOME_&_GARDEN_1' 'HOME_&_GARDEN_2'
 'SUPERMARKET_1' 'SUPERMARKET_2' 'SUPERMARKET_3']


In [80]:
stores = df_sales_original['store'].unique()
print(stores)

['Greenwich_Village' 'Harlem' 'Tribeca' 'Brooklyn' 'South_End' 'Roxbury'
 'Back_Bay' 'Midtown_Village' 'Yorktown' 'Queen_Village']


In [81]:
regions = df_sales_original['region'].unique()
print(regions)

['New York' 'Boston' 'Philadelphia']


#### 3.3. df_price

- Luego de un check ampliando la tabla y viendo "items" aleatorios se puede observar que en la columna yearweek los valores estan ordenados de forma descendente en yearweer y ordenados por item, category y store_code, por lo que se podrian completar las lineas faltantes como fechas consecutivas
- Se podria incluir la columna "sell_price" en el df_sales ya que aqui tambien tenemos "item", "store_code", solo nos faltaria agregar el yearweek en df_sales


In [82]:
df_prices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


In [83]:
df_prices.isnull().value_counts()

item   category  store_code  yearweek  sell_price
False  False     False       False     False         6721786
                             True      False          243920
Name: count, dtype: int64

In [84]:
items = df_prices['item'].unique()
print(items)

['ACCESORIES_1_001' 'ACCESORIES_1_002' 'ACCESORIES_1_003' ...
 'SUPERMARKET_3_825' 'SUPERMARKET_3_826' 'SUPERMARKET_3_827']


In [85]:
pd.set_option('display.max_rows', None)
filtered_df = df_prices[(df_prices['item'] == 'ACCESORIES_1_001')]
filtered_df

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
5,ACCESORIES_1_001,ACCESORIES,NYC_1,201333.0,10.9858
6,ACCESORIES_1_001,ACCESORIES,NYC_1,201334.0,10.9858
7,ACCESORIES_1_001,ACCESORIES,NYC_1,201335.0,10.9858
8,ACCESORIES_1_001,ACCESORIES,NYC_1,201336.0,10.9858
9,ACCESORIES_1_001,ACCESORIES,NYC_1,201337.0,10.9858


In [86]:
filtered_df1 = df_prices[(df_prices['item'] == 'SUPERMARKET_3_827')]
filtered_df1

,item,category,store_code,yearweek,sell_price
711002,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201508.0,1.2
711003,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201509.0,1.2
711004,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201510.0,1.2
711005,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201511.0,1.2
711006,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201512.0,1.2
711007,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201513.0,1.2
711008,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201514.0,1.2
711009,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201515.0,1.2
711010,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201516.0,1.2
711011,SUPERMARKET_3_827,SUPERMARKET,NYC_1,201517.0,1.2


In [87]:
pd.reset_option('display.max_rows')

In [88]:
max = df_prices['sell_price'].max()
expensiveitem = df_prices[df_prices['sell_price'] == max]
expensiveitem

,item,category,store_code,yearweek,sell_price
6604082,HOME_&_GARDEN_2_406,HOME_&_GARDEN,PHI_3,201320.0,134.15
6604083,HOME_&_GARDEN_2_406,HOME_&_GARDEN,PHI_3,201321.0,134.15
6604084,HOME_&_GARDEN_2_406,HOME_&_GARDEN,PHI_3,201322.0,134.15


In [89]:
min = df_prices['sell_price'].min()
cheapitem = df_prices[df_prices['sell_price'] == min]
cheapitem

,item,category,store_code,yearweek,sell_price
2664856,SUPERMARKET_3_413,SUPERMARKET,NYC_4,201152.0,0.012
2664857,SUPERMARKET_3_413,SUPERMARKET,NYC_4,201200.0,0.012
5413301,SUPERMARKET_3_122,SUPERMARKET,PHI_1,201429.0,0.012


## 4. Limpieza de files

### 4.1. Calendar



- Se paso la columna "date" a formato datetime
- Se identifico un problema al intentar crear una columna con el formato "yearweek" ya que utilizando solo la columna date y las formulas genericas *dt.strftime('%Y-%U')* y *dt.isocalendar()* no nos da los resultados esperados, al hacer el merge entre df_sales y df_price para la cual necesitamos el "yearweek" obtenemos NaN.
- Luego de investigacion y pruebas hemos visto que para crear la columna "yearweek" necesitamos una formula personalizada por la cual se crearon columnas "year", "month" y "day", y se hizo la formula a partir de estas columnas.
- Se cambiaron los NaN de la columna "event" por Regular Day
- Se agregaron las fiestas Navidad y 4 de Julio
- Se crearon diccionarios de la columna "d" con "date", "weekday", "event" y "yearweek" para luego incluir estos valores en el df_sales

In [90]:
df_calendar

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,NaN
1909,2016-04-21,Thursday,6,d_1910,NaN
1910,2016-04-22,Friday,7,d_1911,NaN
1911,2016-04-23,Saturday,1,d_1912,NaN


In [91]:
#pasar columna date a formato dtime
df_calendar['date'] = pd.to_datetime(df_calendar['date'])

In [92]:
#Crear columnas Año, mes y dia para poder aplicar la formula year week

df_calendar['year'] = df_calendar['date'].dt.year
df_calendar['month'] = df_calendar['date'].dt.month
df_calendar['day'] = df_calendar['date'].dt.day


In [93]:
# Esta funcion genera la columna yearweek en una calendar_table segun la metodologia de calculo de dsmarket. Hay que introducir tambien
# el año y la semana de la primera fila de la tabla calendar_table.

def get_yearweek(df_calendar,first_year,first_week):
  # inicializo la columna yeraweek como columna de solos 0.
  df_calendar['yearweek'] = 0.
  df_calendar.loc[0,'yearweek'] = float(first_year+first_week)
  week = int(first_week.lstrip('0'))

  for i in range(1,len(df_calendar)):
    year = df_calendar.loc[i,'year']
    month = df_calendar.loc[i,'month']
    day = df_calendar.loc[i,'day']
    weekday_int = df_calendar.loc[i,'weekday_int']
    # es enero y estoy en la ultima yearweek del año anterior, y no es sabado?
    if (month==1) and (week in range(52,55)) and (weekday_int!=1):
      year_str = str(year-1)
      week_str = str(week)
      df_calendar.loc[i,'yearweek']=float(year_str+week_str)
    # es enero y estoy en la ultima yearweek del año anterior, y es sabado?
    elif (month==1) and (week in range(52,55)) and (weekday_int==1):
      week = 1
      week_str = '0'+str(week)
      year_str = str(year)
      df_calendar.loc[i,'yearweek']=float(year_str+week_str)
    # es dicembre y estoy en la semana 51,52 o 53, y no es sabado?
    elif (month==12) and (week in range(51,54)) and (weekday_int!=1):
      year_str = str(year)
      week_str = str(week)
      df_calendar.loc[i,'yearweek']=float(year_str+week_str)
    # es dicembre y estoy en la semana 51,52 o 53, y es sabado?
    elif (month==12) and (week in range(51,54)) and (weekday_int==1):
    # de acuerdo con el dia se pasa a una nueva semana del año viejo o a la primera semana del año nuevo
      if day <= 29:
        year_str = str(year)
        week += 1
        week_str = str(week)
        df_calendar.loc[i,'yearweek']=float(year_str+week_str)
      else:
        if day==30:
          year_str = str(year+1)
          week_str = '01'
          df_calendar.loc[i,'yearweek']=float(year_str+week_str)
        else:
          year_str = str(year+1)
          week = 1
          week_str = '01'
          df_calendar.loc[i,'yearweek']=float(year_str+week_str)
    # En todos los otros casos se empieza una nueva yearweek cada sabado
    # Si no es sabado se sigue en la misma yearweek
    elif weekday_int!=1:
      year_str = str(year)
      if week<10:
        week_str = '0' + str(week)
      else:
        week_str = str(week)
      df_calendar.loc[i,'yearweek']=float(year_str+week_str)
    # Si es sabado se empieza una nueva yearweek
    elif weekday_int==1:
      year_str = str(year)
      week += 1
      if week<10:
        week_str = '0' + str(week)
      else:
        week_str = str(week)
      df_calendar.loc[i,'yearweek']=float(year_str+week_str)

get_yearweek(df_calendar,'2011','05')


In [94]:
df_calendar = df_calendar.fillna('Regular Day')

In [95]:
# formulas para agregar navidad y 4 de julio

df_calendar.loc[(df_calendar['month'] == 12) & (df_calendar['day'] == 25), 'event'] = 'Navidad'

df_calendar.loc[(df_calendar['month'] == 7) & (df_calendar['day'] == 4), 'event'] = '4 de Julio'

In [96]:
df_calendar['yearweek'] = df_calendar['yearweek'].astype(int)
df_calendar['yearweek'] = df_calendar['yearweek'].astype(str)

In [97]:
#crear diccionaron del calendar

map_dict_date = df_calendar.set_index('d')['date'].to_dict()
map_dict_yearweek = df_calendar.set_index('d')['yearweek'].to_dict()
map_dict_event = df_calendar.set_index('d')['event'].to_dict()
map_dict_weekday = df_calendar.set_index('d')['weekday_int'].to_dict()

In [98]:
df_calendar

,date,weekday,weekday_int,d,event,year,month,day,yearweek
0,2011-01-29,Saturday,1,d_1,Regular Day,2011,1,29,201105
1,2011-01-30,Sunday,2,d_2,Regular Day,2011,1,30,201105
2,2011-01-31,Monday,3,d_3,Regular Day,2011,1,31,201105
3,2011-02-01,Tuesday,4,d_4,Regular Day,2011,2,1,201105
4,2011-02-02,Wednesday,5,d_5,Regular Day,2011,2,2,201105
...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,Regular Day,2016,4,20,201616
1909,2016-04-21,Thursday,6,d_1910,Regular Day,2016,4,21,201616
1910,2016-04-22,Friday,7,d_1911,Regular Day,2016,4,22,201616
1911,2016-04-23,Saturday,1,d_1912,Regular Day,2016,4,23,201617


### 4.2. Sales


- Se hizo melt de las columnas "d_" para que sean filas en vez de columnas manteniendo fijas las columas desde "id" a "region"
- Se eliminaron las lineas en las que las ventas (columnas "d_") son 0
- Se resetaron los numeros del index
- Se incluyeron las columnas de "date", "yearweek", "events" y "weekday" a partir de los diccionarios creados anteriormente

In [99]:
df_sales_original

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,2,...,2,0,0,0,0,0,1,0,0,1
30486,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,6,0,...,2,1,0,2,0,1,0,0,1,0
30488,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [100]:
columns_sales = df_sales_original.columns[:7].to_list()
columns_sales

['id', 'item', 'category', 'department', 'store', 'store_code', 'region']

In [101]:
#Mover las ventas a filas
# id_vars = Columnas que no cambian
# var_name = Nombre para la nueva columna de fecha
# value_name = Nombre para los valores de ventas

df_sales = pd.melt(df_sales_original,id_vars=columns_sales,var_name='Fecha',value_name='Qsale')

In [102]:
#eliminar las lineas en las que las ventas son 0

df_sales = df_sales[df_sales['Qsale'] != 0]

In [103]:
#Resetear los numeros del indice

df_sales = df_sales.reset_index(drop=True)

In [104]:
df_sales

,id,item,category,department,store,store_code,region,Fecha,Qsale
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12
1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2
2,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4
3,ACCESORIES_1_016_NYC_1,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,5
4,ACCESORIES_1_022_NYC_1,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2
...,...,...,...,...,...,...,...,...,...
18550271,SUPERMARKET_3_819_PHI_3,SUPERMARKET_3_819,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,2
18550272,SUPERMARKET_3_820_PHI_3,SUPERMARKET_3_820,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
18550273,SUPERMARKET_3_822_PHI_3,SUPERMARKET_3_822,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
18550274,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1


In [105]:
#Agregar fecha, yearweek y event del calendar

df_sales['date'] = df_sales['Fecha'].map(map_dict_date).combine_first(df_sales['Fecha'])
df_sales['yearweek'] = df_sales['Fecha'].map(map_dict_yearweek).combine_first(df_sales['Fecha'])
df_sales['event'] = df_sales['Fecha'].map(map_dict_event).combine_first(df_sales['Fecha'])
df_sales['weekday'] = df_sales['Fecha'].map(map_dict_weekday).combine_first(df_sales['Fecha'])


C:\Users\gaspi\AppData\Local\Temp\ipykernel_23448\2103031368.py:6: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_sales['weekday'] = df_sales['Fecha'].map(map_dict_weekday).combine_first(df_sales['Fecha'])


In [106]:
df_sales

,id,item,category,department,store,store_code,region,Fecha,Qsale,date,yearweek,event,weekday
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,2011-01-29,201105,Regular Day,1
1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,2011-01-29,201105,Regular Day,1
2,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4,2011-01-29,201105,Regular Day,1
3,ACCESORIES_1_016_NYC_1,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,5,2011-01-29,201105,Regular Day,1
4,ACCESORIES_1_022_NYC_1,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,2011-01-29,201105,Regular Day,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550271,SUPERMARKET_3_819_PHI_3,SUPERMARKET_3_819,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,2,2016-04-24,201617,Regular Day,2
18550272,SUPERMARKET_3_820_PHI_3,SUPERMARKET_3_820,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,201617,Regular Day,2
18550273,SUPERMARKET_3_822_PHI_3,SUPERMARKET_3_822,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,201617,Regular Day,2
18550274,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,201617,Regular Day,2


### 4.3. Prices

- Se actualizaron los NaN en la columnas de "yearweek" con una formula personalizada
- Se cambio el formato de la columna "yearweek" para eliminar el .0

In [107]:
df_prices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


In [108]:
#actualizar los nan de las fechas

def fill_yearweek(data):
    prev_yearweek = None  # Variable para llevar el control del valor anterior

    for i in range(len(data)):
        # Si no es NaN, actualizamos el valor de referencia
        if not pd.isna(data[i]):
            prev_yearweek = int(data[i])  # Convertimos a entero
        else:
            # Si es NaN, calculamos el siguiente valor basado en el anterior
            year = prev_yearweek // 100
            week = prev_yearweek % 100 + 1  # Siguiente semana

            # Si la semana excede 52, reiniciamos el año
            if week > 52:
                if year == 2012:
                    week = 53  # Permitir la semana 53 para el año 2012
                else:
                    year += 1
                    week = 1

            prev_yearweek = year * 100 + week
            data[i] = prev_yearweek  # Rellenamos el valor

    return data

# Aplicamos la función dentro de cada grupo de 'item' y 'store_code'
df_prices['yearweek'] = df_prices.groupby(['item', 'store_code'])['yearweek'].transform(lambda x: fill_yearweek(x.to_list()))


In [109]:
#pasar yearweek a str, primero lo paso a int para que no tenga el .0

df_prices['yearweek'] = df_prices['yearweek'].astype(int)
df_prices['yearweek'] = df_prices['yearweek'].astype(str)

In [110]:
df_prices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201621,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201622,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201623,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201624,1.2000


## 5. Merge Prices y Sales

### 5.1. Merge para incluir el sell_price en df_sales

- Se incluye la columna "sell_price" del df_prices en el df_sales a partir de las columnas "item", "store_code" y "yearweek".


In [111]:
df_sales_wprice = df_sales.merge(df_prices[['item','store_code', 'yearweek', 'sell_price']], on=['item','store_code', 'yearweek'], how='left')

### 5.2. Check para ver que no hay NaN de Sell_price luego del merge


- En este punto se verifica que si hay o no NaN al incluir el "sell_price" en el df_sale, si hay NaN es porque la formula de "yearweek" no es correcta, en caso de que haya valores nulos ver si hay algun patron en los dias en los que hay errores
- Este punto nos sirvo para identificar que los errores que teniamos se generaban los ultimos dias del mes de Diciembre en la mayoria de los años

In [112]:
nan_rows_specific = df_sales_wprice[df_sales_wprice['sell_price'].isna()]

In [113]:
nan_rows_specific

,id,item,category,department,store,store_code,region,Fecha,Qsale,date,yearweek,event,weekday,sell_price


In [114]:
nan_rows_specific = df_sales_wprice[df_sales_wprice['sell_price'].isna()]
valores_unicos = nan_rows_specific['date'].unique()
valores_unicos

<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]

## 6. Df_sales_wprice

- Se crear la columna de total_venta en el nuevo df para calcular el total por linea (Qsale*sell_price)
- Se eliminan las columnas "id", "store_code" y "Fecha" ya que no aportan datos de valor y para que el archivo output ocupe menos espacio
- Se crea un nuevo archivo csv a partir de este nuevo df para trabajar en los proximos pasos del proyecto sin la necesidad de ejecturar la limpieza y merge de datos
- Renombrado de columnas
- Se podria cambiar la columna weekday por un flag de dia de fin de semana y cambiar la columna de event por uno de flags ( columnas de 1 y 0)

In [115]:
df_sales_wprice

,id,item,category,department,store,store_code,region,Fecha,Qsale,date,yearweek,event,weekday,sell_price
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,2011-01-29,201105,Regular Day,1,0.6118
1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,2011-01-29,201105,Regular Day,1,2.0748
2,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4,2011-01-29,201105,Regular Day,1,0.9310
3,ACCESORIES_1_016_NYC_1,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,5,2011-01-29,201105,Regular Day,1,0.9310
4,ACCESORIES_1_022_NYC_1,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,2011-01-29,201105,Regular Day,1,9.1238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550271,SUPERMARKET_3_819_PHI_3,SUPERMARKET_3_819,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,2,2016-04-24,201617,Regular Day,2,2.3760
18550272,SUPERMARKET_3_820_PHI_3,SUPERMARKET_3_820,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,201617,Regular Day,2,2.3760
18550273,SUPERMARKET_3_822_PHI_3,SUPERMARKET_3_822,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,201617,Regular Day,2,4.7760
18550274,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,201617,Regular Day,2,3.5760


In [116]:
#Creado de columna total venta Qsale*sell_price
df_sales_wprice['total_venta'] = df_sales_wprice['Qsale']*df_sales_wprice['sell_price']

In [117]:
df_sales_wprice.drop(columns=['Fecha','id','store_code'], inplace=True)

In [118]:
df_sales_wprice.rename(columns={'region': 'city', 'sell_price': 'unit_price'}, inplace=True)

In [119]:
df_sales_wprice

,item,category,department,store,city,Qsale,date,yearweek,event,weekday,unit_price,total_venta
0,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,New York,12,2011-01-29,201105,Regular Day,1,0.6118,7.3416
1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,New York,2,2011-01-29,201105,Regular Day,1,2.0748,4.1496
2,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,New York,4,2011-01-29,201105,Regular Day,1,0.9310,3.7240
3,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,New York,5,2011-01-29,201105,Regular Day,1,0.9310,4.6550
4,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,New York,2,2011-01-29,201105,Regular Day,1,9.1238,18.2476
...,...,...,...,...,...,...,...,...,...,...,...,...
18550271,SUPERMARKET_3_819,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,2,2016-04-24,201617,Regular Day,2,2.3760,4.7520
18550272,SUPERMARKET_3_820,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,1,2016-04-24,201617,Regular Day,2,2.3760,2.3760
18550273,SUPERMARKET_3_822,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,1,2016-04-24,201617,Regular Day,2,4.7760,4.7760
18550274,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,1,2016-04-24,201617,Regular Day,2,3.5760,3.5760


In [120]:
# Formula para extraer el df como archivo csv
#df_sales_wprice.to_csv('sales_wprice.csv', index=False)